In [1]:
import pandas as pd
import numpy as np
import glob, os, sys
from hmmlearn import hmm
osp = sys.platform
# import matplotlib.pyplot as plt
# %matplotlib inline

from utilities import *
from helpers import *
# ---------------------------------------------------------
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
# warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')
# ---------------------------------------------------------
# directories
main_dir = '../'
game_dir = main_dir+'data/'
Data = LoadData(main_dir, game_dir)

In [2]:
# %%time
game_id = '0021500463'
game_data = Data.load_game(game_id)
events_df = pd.DataFrame(game_data['events'])
print('raw events shape:', events_df.shape)
events_df.head(3)

raw events shape: (231, 8)


,end_time_left,home,moments,orig_events,playbyplay,quarter,start_time_left,visitor
0,702.31,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351428029, 708.28, 12.78, None, [[-1,...",[0],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,708.28,"{'abbreviation': 'TOR', 'players': [{'playerid..."
1,686.28,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351428029, 708.28, 12.78, None, [[-1,...",[1],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,708.28,"{'abbreviation': 'TOR', 'players': [{'playerid..."
2,668.42,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351444029, 692.25, 12.21, None, [[-1,...","[2, 3]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,692.25,"{'abbreviation': 'TOR', 'players': [{'playerid..."


In [3]:
# get events and number of events
# events_df = pd.DataFrame(data['events'])
# n_events = events_df.shape[0]
# # get numebr of moments
# moments = events_df.iloc[event_number,:]['moments']
# n_moments = len(moments)

# print(n_moments, event_df.shape)
# event_df.head()

In [4]:
# # get all the player_id and player_name mapping
# player_id_mapping = id_player(event_df)

In [5]:
# # get position mapping
# # get all the player_id and player_name mapping
# position_id_mapping = id_position(event_df)

In [6]:
# set(np.concatenate(list(position_id_mapping.values())))

In [7]:
# play id to play roles/positions
id_role = id_position(events_df)
check_game_roles_duplicates(id_role)

# we will use this fixed order as the role order
roles = ['F', 'G', 'C-F', 'G-F', 'F-G', 'C', 'F-C']
role_order = {'F': 0, 'G':4, 'C-F':1, 'G-F':3, 'F-G':3, 'C':2, 'F-C':1}

# its possible that F has similar role as G-f or F-G, we create empty slots to ensure meta order
# ddentify defending and offending runs (this is included in process_moments)
court_index = Data.load_csv('./meta_data/court_index.csv')
court_index = dict(zip(court_index.game_id, court_index.court_position))

# home and visitor ids
homeid = events_df.loc[0].home['teamid']
awayid = events_df.loc[0].visitor['teamid']

In [8]:
single_game = []
for i in events_df.moments.values:
    result_i = process_moments_ra(i, homeid, awayid, court_index, game_id)
    if result_i == None:
        continue
    else:
        pm, scs = result_i
        d1,d2,d3 = pm.shape
        single_game.append(pm.reshape(d1, d2*d3))
        
seq_lens = np.array([len(i) for i in single_game])
train = np.concatenate(single_game, axis=0)

Warning!: There are less than 10 players! (skip)
Warning!: There are less than 10 players! (skip)
Warning!: There are less than 10 players! (skip)


In [9]:
single_game[0].shape

(92, 20)

In [10]:
train.shape

(35913, 20)

In [11]:
final = []
for i in range(0, train.shape[1], 2):
    final.append(train[:,i:i+2])
final = np.concatenate(final, axis=0)
seq_lens = np.concatenate([seq_lens for _ in range(10)], axis=0)

In [12]:
# Viterbi is set by default as the 'algorithm' optional parameter.
model = hmm.GMMHMM(n_components=6, n_mix=5, verbose=True)#, covariance_type="diag", n_iter=100) 
model.fit(final, lengths=seq_lens)

/home/sam/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/sam/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/home/sam/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/home/sam/anaconda3/lib/python3.6/site-packages/skl

GMMHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
    init_params='stmcw', n_components=6, n_iter=10, n_mix=5,
    params='stmcw', random_state=None, startprob_prior=1.0, tol=0.01,
    transmat_prior=1.0, verbose=True)

In [14]:
state_sequence = model.predict(train, lengths=seq_lens)
# prob_next_step = model.transmat_[state_sequence[-1], :]

ValueError: more than 35913 samples in lengths array [ 92  92 179 ... 140 125 211]

In [ ]:
seq_lens[:10].sum()

In [ ]:
prob_next_step*100

In [ ]:
state_sequence.shape

In [ ]:
state_sequence[:150]

In [ ]:
m = -1
for i in event_df.moments.values:
    for j in i:
        m1 = max(np.array(j[5])[:, 3])
        if m1>m:
            m = m1
m

In [ ]:
event_df.moments.values[0][0][5]

---

### Next: try using the predefined position as indexing mechanism for the imitation learning

Questions:

  - Is the latent structure learning trained for each individual agent or their sequences are put together?